## Neural and N-Gram Language Model for CPSC 503 Assignment 2
#### The neural model notebook is modified from Yunjey Choi's Github repository - pytorch-tutorial.
#### Familiarize yourself with pytorch, start with: https://pytorch.org/tutorials/beginner/basics/intro.html

#### The N-gram model notebook is from Josh Loehr's Github repository - ngram-language-model.
#### https://github.com/joshualoehr/ngram-language-model

#### ========================================================================================

### Let's load a number of dependencies:

In [35]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
import numpy as np
import os
import math
from collections import defaultdict
from tqdm import tqdm

# check if GPU is available to pytorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Here are two classes needed for the data loading and formating:

In [36]:
class Dictionary(object):
    # use to generate and return the word-to-index (index-to-word) vocabulary dictionary
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def return_dict(self):
        return self.idx2word
        
    def __len__(self):
        return len(self.word2idx)


class Corpus(object):
    # load and prepare the corpus  the language models input format
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, n_gram=2):
        with open(path, 'r') as f:
            tokens = 0;
            sample_list = []
            for line in f:
                # add <start> tokens based on the number of n-grams.
                words = ['<start>'] * (n_gram - 1) + line.split() + ['<end>']
                tokens += len(words)
                sample_list.append(words)
                for word in words: 
                    self.dictionary.add_word(word)  

        # # Read corpus and store the each line (word sequence) into its corresponding index sequence.
        ids_list = [[0]*len(s) for s in sample_list if len(s) > n_gram] 
        with open(path, 'r') as f:
            sample_num = 0
            for line in f:
                token = 0
                words = ['<start>'] * (n_gram - 1) + line.split() + ['<end>']
                if len(words) >= n_gram:
                    for word in words:
                        ids_list[sample_num][token] = self.dictionary.word2idx[word]
                        token += 1
                    sample_num += 1

        # FOR THE NEURAL MODEL Convert the flat index sequences into the n-gram tensors which are ready for neural model.
        for n in range(len(ids_list)):
            flat_ids = ids_list[n]
            ids_list[n] = torch.LongTensor([flat_ids[i:i+n_gram] for i in range(len(flat_ids)-(n_gram - 1))])
        return ids_list

### Hyper-parameters for both Language Models: 

In [37]:
# Bigram model
n_gram = 2

# m_gram is the number of preceding/conditioning words 
m_gram = n_gram - 1

In [78]:
corpus = Corpus()
ids = corpus.get_data('data/train_mini.txt', n_gram)
# ids = corpus.get_data('data/train.txt', n_gram)


# Use 70% for training, 15% for development, and 15% for testing 
n_train = round(len(ids) * .75)
n_dev = round(len(ids) * .15)

train_ids = ids[:n_train]
dev_ids = ids[n_train:n_train + n_dev]
test_ids = ids[n_train + n_dev:]

print(f"Number of sentences: {len(train_ids)} train, {len(dev_ids)} dev, {len(test_ids)} test")
vocab_size = len(corpus.dictionary)
print(f"Vocab size: {vocab_size}")

Number of sentences: 786 train, 157 dev, 105 test
Vocab size: 3580


### Load the "Penn Treebank" dataset and split it into train/dev/test

### The class of count-based language model:

In [79]:
class CountLM(object):
    def __init__(self, vocab_size, x_n, x_m, laplace=1):
        self.vocab_size = vocab_size
        self.laplace = laplace
        
        # Dictionaries for tracking the count of n-grams
        self.n_gram_count = self.count_ngrams(x_n)
        self.m_gram_count = self.count_ngrams(x_m)
    
    def count_ngrams(self, x):
        """
        Populate the dictionary with the number of occurrences of each n-gram
        """
        count_list = defaultdict(int)
        for example in x:
            for n_gram in example.tolist():
                count_list[tuple(n_gram)] += 1 
                
        return count_list
    
    def compute_mle(self, n_gram):
        """
        Compute the MLE of P(w_n|w_{n−1}, ...) with add-one Laplacian smoothing
        
        Please see chapter 3.5.1 of J&M 3rd Ed. for more information
        """
        n_count = self.n_gram_count[n_gram]
        
        m_gram = n_gram[:-1]
        m_count = self.m_gram_count[m_gram]
        prob = (n_count + self.laplace) / (m_count + self.laplace * self.vocab_size) 
        return prob


### Train the n-gram model based on MLE

In [103]:
m_gram_train_ids = corpus.get_data('data/train_mini.txt', m_gram)[:-200]
# m_gram_train_ids = corpus.get_data('data/train.txt', m_gram)[:n_train]
n_gram_train_ids = train_ids

# Populate the dictionaries with counts from training corpus
count_model = CountLM(vocab_size, n_gram_train_ids, m_gram_train_ids)

# Compute average perplexity on training set
train_ppl = 0
for i in range(0, len(train_ids)):
    
    probabilities = list(map(lambda x: count_model.compute_mle(tuple(x)), train_ids[i].tolist()))
    perplexity = np.exp(sum(-np.log(probabilities)) / len(train_ids[i]))
    train_ppl += perplexity
#     print(probabilities)
print('The average training perplexity for count-based LM: '+str(train_ppl/len(train_ids)))

The average training perplexity for count-based LM: 1026.5023506565244


In [92]:
we = corpus.dictionary.word2idx["we"]
want = corpus.dictionary.word2idx["want"]
count_model.compute_mle((we, want))
# count_model.n_gram_count

0.000278473962684489

conditional prob of the word given the previous context
get all the words in the vocab...
given train IDs how do I map it to a word?
How do I loop through all the words and get all the probabilities?
normalize the sum of the count to 1

In [153]:
the = corpus.dictionary.word2idx["the"]
zero = corpus.dictionary.idx2word[0]
count_model.compute_mle((0, the))
x = count_model.n_gram_count

In [168]:
the = corpus.dictionary.word2idx["the"]
zero = corpus.dictionary.idx2word[0]
count_model.compute_mle((0, the))
x = count_model.n_gram_count
while True:
    value = 0
    key
    for pair in x:
    #     print(pair)
        try:
            prob = count_model.compute_mle(pair)
            if prob > value:
                value = prob
                key = pair

        except TypeError:
            # hit end of dict
            
            break
    prob = count_model.compute_mle(key)
    x = count_model.n_gram_count
    print(prob)
    
    
# print(key, value)
# print(corpus.dictionary.idx2word[key[1]])
# prob = count_model.compute_mle(key)
# print(prob)

0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665
0.03910614525139665


KeyboardInterrupt: 

In [8]:
# Compute average perplexity on testing set
test_ppl = 0
for i in range(0, len(test_ids)):

    probabilities = list(map(lambda x: count_model.compute_mle(tuple(x)), test_ids[i].tolist()))
    perplexity = np.exp(sum(-np.log(probabilities)) / len(test_ids[i]))
    print('Perplexity for test sample '+str(i)+' :', perplexity)
    test_ppl += perplexity
    
print('The average testing perplexity for count-based LM: '+str(test_ppl/len(test_ids)))

Perplexity for test sample 0 : 481.93397924150236
Perplexity for test sample 1 : 1791.0646372289625
Perplexity for test sample 2 : 295.4281859753384
Perplexity for test sample 3 : 336.8972086530059
Perplexity for test sample 4 : 188.3063912073697
Perplexity for test sample 5 : 1580.4694541715269
Perplexity for test sample 6 : 908.3518812063347
Perplexity for test sample 7 : 587.567516576945
Perplexity for test sample 8 : 466.1199712311848
Perplexity for test sample 9 : 710.2569626653177
Perplexity for test sample 10 : 3201.2726581592956
Perplexity for test sample 11 : 731.7403780164503
Perplexity for test sample 12 : 902.777634164664
Perplexity for test sample 13 : 2107.650302714663
Perplexity for test sample 14 : 867.2708592238805
Perplexity for test sample 15 : 1282.4375182196761
Perplexity for test sample 16 : 516.5791519391194
Perplexity for test sample 17 : 1637.3482160822728
Perplexity for test sample 18 : 891.4693914511101
Perplexity for test sample 19 : 1841.6775228827773
Perpl

Perplexity for test sample 522 : 383.2279527359837
Perplexity for test sample 523 : 1163.9777024650646
Perplexity for test sample 524 : 465.934322506438
Perplexity for test sample 525 : 1334.0055729360536
Perplexity for test sample 526 : 2180.096615174918
Perplexity for test sample 527 : 1108.7488559401008
Perplexity for test sample 528 : 462.42303691332467
Perplexity for test sample 529 : 1829.5391982113467
Perplexity for test sample 530 : 680.7873439128163
Perplexity for test sample 531 : 1075.720921572503
Perplexity for test sample 532 : 1439.8581346877356
Perplexity for test sample 533 : 774.3979944399077
Perplexity for test sample 534 : 703.7149869261153
Perplexity for test sample 535 : 1840.1099473425838
Perplexity for test sample 536 : 1048.4176752574454
Perplexity for test sample 537 : 1971.689039941437
Perplexity for test sample 538 : 1139.6771184979086
Perplexity for test sample 539 : 472.15129462701213
Perplexity for test sample 540 : 269.29719943722847
Perplexity for test s

Perplexity for test sample 1013 : 1711.7082329941663
Perplexity for test sample 1014 : 379.5192028280277
Perplexity for test sample 1015 : 2401.26656732838
Perplexity for test sample 1016 : 843.8697037125777
Perplexity for test sample 1017 : 284.7970686294181
Perplexity for test sample 1018 : 798.2743772249205
Perplexity for test sample 1019 : 454.1174549175305
Perplexity for test sample 1020 : 422.26318313376515
Perplexity for test sample 1021 : 685.499143250206
Perplexity for test sample 1022 : 844.0352499114954
Perplexity for test sample 1023 : 1612.4572589188404
Perplexity for test sample 1024 : 1360.7913047868365
Perplexity for test sample 1025 : 2522.872881530421
Perplexity for test sample 1026 : 1094.4399862834046
Perplexity for test sample 1027 : 508.29628617229054
Perplexity for test sample 1028 : 1106.0300800743587
Perplexity for test sample 1029 : 948.5602366707981
Perplexity for test sample 1030 : 770.1203563949285
Perplexity for test sample 1031 : 1801.8496264164366
Perple

Perplexity for test sample 1531 : 577.1488602828803
Perplexity for test sample 1532 : 299.9255184957937
Perplexity for test sample 1533 : 827.8076804668558
Perplexity for test sample 1534 : 705.3064289852103
Perplexity for test sample 1535 : 663.849079561899
Perplexity for test sample 1536 : 711.7755184372891
Perplexity for test sample 1537 : 2288.6995492379706
Perplexity for test sample 1538 : 402.1920779200347
Perplexity for test sample 1539 : 374.4837679885483
Perplexity for test sample 1540 : 301.62385188074904
Perplexity for test sample 1541 : 268.5909645429137
Perplexity for test sample 1542 : 480.63808049208467
Perplexity for test sample 1543 : 1405.4494167726066
Perplexity for test sample 1544 : 1233.6836836396255
Perplexity for test sample 1545 : 964.158641688546
Perplexity for test sample 1546 : 2632.0574769360564
Perplexity for test sample 1547 : 736.6576720700164
Perplexity for test sample 1548 : 768.7408603227124
Perplexity for test sample 1549 : 1089.5486884190204
Perplex

Perplexity for test sample 2043 : 861.2297931704337
Perplexity for test sample 2044 : 110.09632460794144
Perplexity for test sample 2045 : 140.54405500883854
Perplexity for test sample 2046 : 1048.592085992945
Perplexity for test sample 2047 : 612.4826962422168
Perplexity for test sample 2048 : 946.1144442717643
Perplexity for test sample 2049 : 370.91632651931025
Perplexity for test sample 2050 : 255.6946883265172
Perplexity for test sample 2051 : 271.5976210243736
Perplexity for test sample 2052 : 322.3281938989347
Perplexity for test sample 2053 : 423.3829105781211
Perplexity for test sample 2054 : 495.4090954098413
Perplexity for test sample 2055 : 187.84196154116012
Perplexity for test sample 2056 : 683.4962631886007
Perplexity for test sample 2057 : 503.32404625354593
Perplexity for test sample 2058 : 1276.2226596241794
Perplexity for test sample 2059 : 634.3246230358413
Perplexity for test sample 2060 : 382.6625850446478
Perplexity for test sample 2061 : 3283.5156372008087
Perpl

Perplexity for test sample 2579 : 605.0602644009936
Perplexity for test sample 2580 : 1442.6644536325182
Perplexity for test sample 2581 : 182.68606183602927
Perplexity for test sample 2582 : 425.6359296698872
Perplexity for test sample 2583 : 748.6310597242529
Perplexity for test sample 2584 : 1141.9374107897643
Perplexity for test sample 2585 : 820.109041897057
Perplexity for test sample 2586 : 732.5077261378736
Perplexity for test sample 2587 : 1582.8616487707743
Perplexity for test sample 2588 : 1028.554216184425
Perplexity for test sample 2589 : 977.2569026619584
Perplexity for test sample 2590 : 1513.6263758858752
Perplexity for test sample 2591 : 214.30865853296692
Perplexity for test sample 2592 : 4331.1556271852505
Perplexity for test sample 2593 : 1030.9518452066873
Perplexity for test sample 2594 : 2282.1208493960876
Perplexity for test sample 2595 : 248.37353419604037
Perplexity for test sample 2596 : 381.4080662096307
Perplexity for test sample 2597 : 550.7581864087533
Per

Perplexity for test sample 3028 : 1261.8112982701539
Perplexity for test sample 3029 : 2006.224132054
Perplexity for test sample 3030 : 1739.668878912662
Perplexity for test sample 3031 : 2617.620305319237
Perplexity for test sample 3032 : 1103.0667006056615
Perplexity for test sample 3033 : 727.801165168394
Perplexity for test sample 3034 : 113.86124725832116
Perplexity for test sample 3035 : 373.8212565838409
Perplexity for test sample 3036 : 974.4019105063632
Perplexity for test sample 3037 : 298.32207411577417
Perplexity for test sample 3038 : 492.3418672006072
Perplexity for test sample 3039 : 443.67791797312304
Perplexity for test sample 3040 : 2408.3122701450493
Perplexity for test sample 3041 : 460.6088781904691
Perplexity for test sample 3042 : 1301.0712495012476
Perplexity for test sample 3043 : 485.5108436196283
Perplexity for test sample 3044 : 201.9283952336438
Perplexity for test sample 3045 : 955.7219220031121
Perplexity for test sample 3046 : 1202.7081907056463
Perplexi

Perplexity for test sample 3530 : 997.4662625708547
Perplexity for test sample 3531 : 451.1659927828725
Perplexity for test sample 3532 : 297.71909643495354
Perplexity for test sample 3533 : 872.0652912531395
Perplexity for test sample 3534 : 516.204174117343
Perplexity for test sample 3535 : 1377.026592717125
Perplexity for test sample 3536 : 1020.3817831975803
Perplexity for test sample 3537 : 5196.320377062466
Perplexity for test sample 3538 : 1718.4717987308995
Perplexity for test sample 3539 : 484.7088777285546
Perplexity for test sample 3540 : 416.3101844030728
Perplexity for test sample 3541 : 1025.1644117532715
Perplexity for test sample 3542 : 633.4810840069381
Perplexity for test sample 3543 : 647.7869467597407
Perplexity for test sample 3544 : 1924.3657481362886
Perplexity for test sample 3545 : 1700.785021048634
Perplexity for test sample 3546 : 2640.525586784412
Perplexity for test sample 3547 : 1823.0024346119217
Perplexity for test sample 3548 : 1126.1181619967729
Perple

Perplexity for test sample 4021 : 82.65844465400002
Perplexity for test sample 4022 : 282.59626961095927
Perplexity for test sample 4023 : 270.44256557604655
Perplexity for test sample 4024 : 370.74600106092873
Perplexity for test sample 4025 : 284.59926331014435
Perplexity for test sample 4026 : 818.6174979486494
Perplexity for test sample 4027 : 855.6481067946715
Perplexity for test sample 4028 : 1104.6814809002547
Perplexity for test sample 4029 : 2221.80807749992
Perplexity for test sample 4030 : 1088.9258127562407
Perplexity for test sample 4031 : 2101.7298594087565
Perplexity for test sample 4032 : 666.4617562975096
Perplexity for test sample 4033 : 323.15513871994847
Perplexity for test sample 4034 : 36.08325561884498
Perplexity for test sample 4035 : 47.01492395478094
Perplexity for test sample 4036 : 644.1534902951437
Perplexity for test sample 4037 : 218.29294127090128
Perplexity for test sample 4038 : 846.3648158499624
Perplexity for test sample 4039 : 127.3814455523921
Perp

### Hyper-parameters for the neural language model

In [14]:
# FOR THE NEURAL MODEL
embed_size = 128
intermediate_size = 1024
num_epochs = 3
learning_rate = 1e-1

### The class for the neural language model:

In [69]:
class NeuralLM(nn.Module):
    def __init__(self, vocab_size, embed_size, intermediate_size, m_gram):
        super(NeuralLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.intermediate = nn.Linear(m_gram * embed_size, intermediate_size)
        self.final = nn.Linear(intermediate_size, vocab_size)
        
    def forward(self, x):
        x = self.embed(x) # Embed word id(s) to vectors
        print(x)
        conc_emb = x.view(x.size(0), x.size(1)*x.size(2))
        intermediate_output = self.intermediate(conc_emb) # one layer of MLP
        intermediate_output = F.relu(intermediate_output) # ReLU non-linear function
        final_out = self.final(intermediate_output) # Map to the vocabulary size output
        return final_out
    
    # x is the context vector and output is the prob distrbution over all the tokens
    # use softmax to make everything into 1
    # choose word with highest probability
    # at each step make the modl take in the context x again that was generated previously
    # apply softmax to create a conditional distribution and take the argmax from that
    # do this iteratively until reaching the end of the sentence

In [70]:
neural_model = NeuralLM(vocab_size, embed_size, intermediate_size, m_gram).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(neural_model.parameters(), lr=learning_rate, momentum=0.9)

### Train the neural model

In [71]:
# Reduce batch size if you are running out of memory
batch_size = 64
training_data = torch.cat(train_ids, dim=0)
neural_model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for i in tqdm(range(0, len(training_data), batch_size)):
        batch = training_data[i:i + batch_size]
        inputs = batch[:, 0:n_gram-1].to(device)
        targets = batch[:, n_gram-1:].to(device)
        
        # Forward pass
        outputs = neural_model(inputs)
        loss = criterion(outputs, targets.reshape(-1))
        total_loss += loss.item();
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    # Calculate the performance (perplexity) of the current trained model on dev set.
    total_ppl = 0
    for i in range(0, len(dev_ids)):
        dev_inputs = dev_ids[i][:, 0:n_gram-1].to(device)
        dev_targets = dev_ids[i][:, n_gram-1:].to(device)
        dev_outputs = neural_model(dev_inputs)
        ce = criterion(dev_outputs, dev_targets.reshape(-1))
        total_ppl += np.exp(ce.item());
    
    print ('Epoch [{}/{}], Training Loss: {:.4f}, Dev Perplexity: {:5.2f}'
        .format(epoch + 1, num_epochs, total_loss/len(train_ids), total_ppl/len(dev_ids)))

  0%|          | 1/10910 [00:00<27:34,  6.59it/s]

tensor([[[ 0.3366, -0.6849, -0.6231,  ...,  0.4395,  0.2915,  0.8121]],

        [[-2.9914,  1.7914, -0.2272,  ..., -0.8930,  0.2622,  0.6680]],

        [[ 0.2563,  0.6231,  0.0235,  ..., -0.1732,  1.9910,  0.6156]],

        ...,

        [[ 0.9145, -0.7784,  0.3912,  ..., -1.1965,  1.0594,  1.0600]],

        [[-0.4304,  0.2398,  0.0237,  ..., -0.3762,  0.7208, -0.1786]],

        [[ 0.2462,  0.7818, -0.2613,  ...,  0.8993,  0.1481, -0.9627]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[ 1.7959,  0.6012, -0.4454,  ...,  0.0339, -1.8028,  1.4657]],

        [[-0.8503,  1.5785,  0.5406,  ...,  0.0375, -0.0828, -0.4686]],

        [[-0.2936,  0.5210,  1.4298,  ...,  0.2086, -0.4422,  0.9478]],

        ...,

        [[ 0.3705, -0.4952,  0.4017,  ...,  0.6291,  0.8509, -1.6369]],

        [[ 0.6665, -0.9370,  1.3804,  ..., -1.2028, -0.0238,  1.3010]],

        [[ 0.4892,  0.5927, -0.0739,  ...,  0.6776, -0.3975, -1.1005]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[ 0.2697,  0.

  0%|          | 3/10910 [00:00<23:57,  7.59it/s]


tensor([[[-1.7487, -0.1242,  0.7775,  ..., -1.2465, -1.2272,  0.9909]],

        [[-1.1443, -0.5819, -0.2509,  ..., -0.3759, -0.4278,  0.4879]],

        [[-1.4152, -1.7207, -0.1968,  ...,  0.7618, -0.8367,  0.6615]],

        ...,

        [[ 0.3361, -0.6847, -0.6231,  ...,  0.4395,  0.2912,  0.8119]],

        [[-0.1149, -1.3540, -1.8638,  ..., -0.1145,  0.6369, -0.8028]],

        [[-2.1575,  0.1363,  0.5428,  ...,  0.4330, -0.2222,  0.1070]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[ 0.3274, -2.0989,  1.3299,  ..., -0.2913,  1.3246,  0.8078]],

        [[ 1.0870, -0.6926, -1.3763,  ..., -0.2518,  0.2096, -1.1757]],

        [[-0.3789,  1.2668,  1.8092,  ...,  0.7507, -0.2402,  1.1666]],

        ...,

        [[-1.3094,  0.0925,  0.8392,  ...,  0.5384, -1.1800,  0.1376]],

        [[-0.1798,  0.7391, -1.0055,  ..., -0.8215, -0.2188,  1.4452]],

        [[-0.8567,  0.5846,  1.2398,  ..., -0.6659,  0.1959, -0.1028]]],
       grad_fn=<EmbeddingBackward0>)


  0%|          | 7/10910 [00:00<20:02,  9.07it/s]

...

tensor([[[ 0.1295, -0.4721, -0.4598,  ..., -1.5921,  0.4118, -1.0547]],

        [[ 1.6932,  1.2153, -0.0809,  ..., -1.7524,  0.4872,  0.4760]],

        [[-0.5364, -0.6313,  0.1116,  ...,  0.1265,  2.2160, -1.0680]],

        ...,

        [[-1.4719,  0.1132,  0.9571,  ...,  2.1825, -1.8582,  2.3388]],

        [[-1.5534, -0.2927, -0.9262,  ..., -0.0173, -0.1407, -1.5474]],

        [[-0.4508,  0.2269,  0.0307,  ..., -0.3738,  0.7086, -0.1671]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[-1.2912,  0.0811,  0.8183,  ...,  0.5466, -1.1683,  0.1394]],

        [[-0.8891, -0.8233,  0.7832,  ...,  0.1050, -0.1467, -0.3869]],

        [[-2.2781,  1.4609, -0.1848,  ..., -1.5307,  0.7403, -0.0354]],

        ...,

        [[-0.4504,  0.2273,  0.0308,  ..., -0.3733,  0.7087, -0.1665]],

        [[ 0.9734,  0.8317, -0.4412,  ..., -0.5247,  1.1247, -0.4153]],

        [[-0.4180,  0.8064, -1.0305,  ..., -1.3214, -0.6060, -1.1334]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[-0.7044,  1.

  1%|          | 103/10910 [00:09<15:53, 11.34it/s]

tensor([[[ 0.3691,  0.7627, -0.4301,  ...,  2.0937,  0.2941,  0.3334]],

        [[ 1.4136,  0.7540,  1.2608,  ..., -2.3383,  2.1072,  0.6525]],

        [[ 0.7863,  0.9849,  1.2730,  ...,  0.8301, -1.0789,  0.0075]],

        ...,

        [[-0.2370,  0.0427,  0.7077,  ..., -0.4675, -0.1689,  1.9536]],

        [[-1.2906,  0.0818,  0.8184,  ...,  0.5471, -1.1675,  0.1399]],

        [[ 0.4523,  1.4972,  0.9120,  ...,  0.4059,  1.1235, -0.3198]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[-1.0260, -0.4982,  0.5264,  ...,  0.1421, -1.4814,  0.0974]],

        [[-1.3131,  1.1988, -0.2671,  ..., -0.2855,  1.0091,  0.8064]],

        [[-0.1530,  0.4134, -2.1864,  ..., -0.4470,  0.5353,  0.6312]],

        ...,

        [[ 0.3407, -0.5136,  0.1256,  ...,  0.4515,  0.0081,  0.2883]],

        [[-0.1533,  1.1557, -0.9706,  ..., -0.5088,  0.0134,  0.3478]],

        [[-0.6179, -1.4169,  2.1333,  ...,  2.1228,  1.5674,  0.3188]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[ 0.2752,  1.

  1%|          | 104/10910 [00:09<16:19, 11.03it/s]

KeyboardInterrupt: 

### Save the trained model

In [13]:
torch.save(neural_model, 'model.ckpt')

### Neural Model testing

In [17]:
neural_model = torch.load('model.ckpt')
neural_model.eval()
test_ppl = 0
with torch.no_grad():
    for i in range(0, len(test_ids)):
        inputs = test_ids[i][:, 0:n_gram-1].to(device)
        gold = test_ids[i][:, n_gram-1:].to(device)
        output = neural_model(inputs)
        cross_entropy = criterion(output, gold.reshape(-1)).item()
        print('Perplexity for test sample '+str(i)+' :', np.exp(cross_entropy))
        test_ppl += np.exp(cross_entropy)
    print('The average testing perplexity for neural LM: '+str(test_ppl/len(test_ids)))

Perplexity for test sample 0 : 131.72969878621925
Perplexity for test sample 1 : 1574.7212829407829
Perplexity for test sample 2 : 218.10068995888324
Perplexity for test sample 3 : 120.06119094099284
Perplexity for test sample 4 : 85.2189484101646
Perplexity for test sample 5 : 515.2834415076813
Perplexity for test sample 6 : 371.81643851171054
Perplexity for test sample 7 : 287.75292782528777
Perplexity for test sample 8 : 119.88790696081533
Perplexity for test sample 9 : 356.07168889966863
Perplexity for test sample 10 : 1160.8481994215201
Perplexity for test sample 11 : 279.610504443154
Perplexity for test sample 12 : 206.2456031778053
Perplexity for test sample 13 : 682.8737300757066
Perplexity for test sample 14 : 440.3478334698897
Perplexity for test sample 15 : 452.6268063150043
Perplexity for test sample 16 : 168.31670265709465
Perplexity for test sample 17 : 1125.5227618518334
Perplexity for test sample 18 : 843.6833387365839
Perplexity for test sample 19 : 551.4993537902428
P

Perplexity for test sample 179 : 390.15148680103533
Perplexity for test sample 180 : 257.0361527765083
Perplexity for test sample 181 : 208.62771096613014
Perplexity for test sample 182 : 117.51346527575234
Perplexity for test sample 183 : 173.71215614960136
Perplexity for test sample 184 : 699.3643507298634
Perplexity for test sample 185 : 402.58037402664274
Perplexity for test sample 186 : 292.3491922671582
Perplexity for test sample 187 : 203.49547878925674
Perplexity for test sample 188 : 317.8527091221824
Perplexity for test sample 189 : 292.04447691064973
Perplexity for test sample 190 : 438.7068008240047
Perplexity for test sample 191 : 372.5351806726931
Perplexity for test sample 192 : 379.72485554992653
Perplexity for test sample 193 : 89.87452370040987
Perplexity for test sample 194 : 816.3362612784208
Perplexity for test sample 195 : 75.12929250008855
Perplexity for test sample 196 : 1583.7841712815425
Perplexity for test sample 197 : 19.003800158585406
Perplexity for test s

Perplexity for test sample 344 : 299.98657862785797
Perplexity for test sample 345 : 516.7430345591669
Perplexity for test sample 346 : 459.70466007034145
Perplexity for test sample 347 : 155.76774282418833
Perplexity for test sample 348 : 416.55883886160694
Perplexity for test sample 349 : 131.0485562675835
Perplexity for test sample 350 : 309.52688560125193
Perplexity for test sample 351 : 808.0927779041558
Perplexity for test sample 352 : 381.5454885471305
Perplexity for test sample 353 : 417.8623025283808
Perplexity for test sample 354 : 42.31317870548912
Perplexity for test sample 355 : 837.4211662048051
Perplexity for test sample 356 : 47.72140544733154
Perplexity for test sample 357 : 295.66585996151775
Perplexity for test sample 358 : 558.8113077468736
Perplexity for test sample 359 : 451.13615935025337
Perplexity for test sample 360 : 348.5721083715625
Perplexity for test sample 361 : 431.1408609375156
Perplexity for test sample 362 : 368.61802747261174
Perplexity for test sam

Perplexity for test sample 506 : 456.9742818824559
Perplexity for test sample 507 : 518.1988549099124
Perplexity for test sample 508 : 902.4547593359952
Perplexity for test sample 509 : 234.34105931101485
Perplexity for test sample 510 : 607.9692481038168
Perplexity for test sample 511 : 682.8470297930626
Perplexity for test sample 512 : 219.69988562656167
Perplexity for test sample 513 : 739.3627804507234
Perplexity for test sample 514 : 259.10004470337
Perplexity for test sample 515 : 270.64542629295744
Perplexity for test sample 516 : 321.7170976750267
Perplexity for test sample 517 : 802.2225281645957
Perplexity for test sample 518 : 242.91464185022613
Perplexity for test sample 519 : 265.21558968793096
Perplexity for test sample 520 : 370.4809677913031
Perplexity for test sample 521 : 487.4680354600938
Perplexity for test sample 522 : 86.67060736792448
Perplexity for test sample 523 : 304.623804404759
Perplexity for test sample 524 : 184.02501108140643
Perplexity for test sample 5

Perplexity for test sample 666 : 115.84913894587257
Perplexity for test sample 667 : 280.91107450601083
Perplexity for test sample 668 : 1054.4221762685115
Perplexity for test sample 669 : 308.18331078297496
Perplexity for test sample 670 : 288.7085411815176
Perplexity for test sample 671 : 476.0244267501216
Perplexity for test sample 672 : 816.1295906692438
Perplexity for test sample 673 : 228.77451633018677
Perplexity for test sample 674 : 910.223076708887
Perplexity for test sample 675 : 307.7988272005231
Perplexity for test sample 676 : 276.9513232878786
Perplexity for test sample 677 : 549.2410868161959
Perplexity for test sample 678 : 404.8303282170591
Perplexity for test sample 679 : 331.2383892947579
Perplexity for test sample 680 : 73.37195391165386
Perplexity for test sample 681 : 86.0901277751469
Perplexity for test sample 682 : 162.61188550778746
Perplexity for test sample 683 : 481.96693822259925
Perplexity for test sample 684 : 510.7924506440719
Perplexity for test sample

...

Perplexity for test sample 3456 : 346.96969506160514
Perplexity for test sample 3457 : 779.5350133034922
Perplexity for test sample 3458 : 215.9938343651339
Perplexity for test sample 3459 : 369.1668404254327
Perplexity for test sample 3460 : 383.55971358292027
Perplexity for test sample 3461 : 542.4678114097662
Perplexity for test sample 3462 : 382.5162533824486
Perplexity for test sample 3463 : 922.3575135277409
Perplexity for test sample 3464 : 341.338215911939
Perplexity for test sample 3465 : 854.8744602085543
Perplexity for test sample 3466 : 305.95828017407763
Perplexity for test sample 3467 : 433.19004249426155
Perplexity for test sample 3468 : 496.0385249459678
Perplexity for test sample 3469 : 258.6098976928039
Perplexity for test sample 3470 : 297.3079376587382
Perplexity for test sample 3471 : 174.56243422190045
Perplexity for test sample 3472 : 576.7277837485801
Perplexity for test sample 3473 : 583.066307435759
Perplexity for test sample 3474 : 1537.4897924876639
Perplexi

Perplexity for test sample 3615 : 396.31154136019666
Perplexity for test sample 3616 : 361.9068961458811
Perplexity for test sample 3617 : 417.61370966709814
Perplexity for test sample 3618 : 464.0913489746982
Perplexity for test sample 3619 : 143.02328513326034
Perplexity for test sample 3620 : 200.90346061306929
Perplexity for test sample 3621 : 168.06167035108956
Perplexity for test sample 3622 : 112.90241665426706
Perplexity for test sample 3623 : 432.53841966508634
Perplexity for test sample 3624 : 332.9476617767729
Perplexity for test sample 3625 : 97.37824134067287
Perplexity for test sample 3626 : 409.9833786957401
Perplexity for test sample 3627 : 992.0193402375203
Perplexity for test sample 3628 : 103.21691184419983
Perplexity for test sample 3629 : 624.6145520302589
Perplexity for test sample 3630 : 606.6594328414052
Perplexity for test sample 3631 : 458.94990244901703
Perplexity for test sample 3632 : 495.61862619872005
Perplexity for test sample 3633 : 191.64538393553008
P

Perplexity for test sample 3781 : 151.22478267162074
Perplexity for test sample 3782 : 62.107731824086265
Perplexity for test sample 3783 : 58.02876495725955
Perplexity for test sample 3784 : 95.23843694852437
Perplexity for test sample 3785 : 258.57611164239927
Perplexity for test sample 3786 : 108.34657958744512
Perplexity for test sample 3787 : 43.941568201059354
Perplexity for test sample 3788 : 65.31284525411203
Perplexity for test sample 3789 : 1149.841075829033
Perplexity for test sample 3790 : 158.38536012183388
Perplexity for test sample 3791 : 97.51917622866702
Perplexity for test sample 3792 : 165.56850117786917
Perplexity for test sample 3793 : 649.4415518187487
Perplexity for test sample 3794 : 92.00636307305375
Perplexity for test sample 3795 : 144.30822806189587
Perplexity for test sample 3796 : 94.34227188360477
Perplexity for test sample 3797 : 199.83519082428762
Perplexity for test sample 3798 : 127.05302159255983
Perplexity for test sample 3799 : 408.62401246034005
P

Perplexity for test sample 3944 : 353.82014504386063
Perplexity for test sample 3945 : 134.9482017539889
Perplexity for test sample 3946 : 295.0333761797733
Perplexity for test sample 3947 : 151.39072572189576
Perplexity for test sample 3948 : 48.84482859705798
Perplexity for test sample 3949 : 110.63604046926572
Perplexity for test sample 3950 : 80.83085649226966
Perplexity for test sample 3951 : 693.3787491738024
Perplexity for test sample 3952 : 252.95032141914848
Perplexity for test sample 3953 : 531.1965902902167
Perplexity for test sample 3954 : 489.8872405899352
Perplexity for test sample 3955 : 143.8103429957771
Perplexity for test sample 3956 : 398.57587153755344
Perplexity for test sample 3957 : 302.142534987776
Perplexity for test sample 3958 : 199.4078966807271
Perplexity for test sample 3959 : 310.4246705781878
Perplexity for test sample 3960 : 359.0709949087227
Perplexity for test sample 3961 : 1980.321303905847
Perplexity for test sample 3962 : 157.11840347627768
Perplex

Perplexity for test sample 4106 : 652.1382357429038
Perplexity for test sample 4107 : 213.60511778514325
Perplexity for test sample 4108 : 309.3934894500949
Perplexity for test sample 4109 : 291.1590291526794
Perplexity for test sample 4110 : 281.22616480733814
Perplexity for test sample 4111 : 81.70808386257194
Perplexity for test sample 4112 : 142.81713242873408
Perplexity for test sample 4113 : 285.51729267336304
Perplexity for test sample 4114 : 197.62150060104045
Perplexity for test sample 4115 : 121.58503325319178
Perplexity for test sample 4116 : 64.5318923954727
Perplexity for test sample 4117 : 508.689226262845
Perplexity for test sample 4118 : 95.4068871579686
Perplexity for test sample 4119 : 47.46131784363054
Perplexity for test sample 4120 : 199.78002619883685
Perplexity for test sample 4121 : 74.93787601807989
Perplexity for test sample 4122 : 11.752772323634925
Perplexity for test sample 4123 : 254.7890100295439
Perplexity for test sample 4124 : 680.7195796483549
Perplex

In [170]:
neural_model(the)

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not int

In [57]:
    value = 0
    key
    for pair in x:
    #     print(pair)
        try:
            prob = neural_model.compute_mle(pair)
            if prob > value:
                value = prob
                key = pair

        except TypeError:
            # hit end of dict
            
            break
    prob = count_model.compute_mle(key)
    x = count_model.n_gram_count
    print(prob)
    
    
    for i in range(0, len(test_ids)):
        inputs = test_ids[i][:, 0:n_gram-1].to(device)
        gold = test_ids[i][:, n_gram-1:].to(device)
        output = neural_model(inputs)
        cross_entropy = criterion(output, gold.reshape(-1)).item()
        print('Perplexity for test sample '+str(i)+' :', np.exp(cross_entropy))
        test_ppl += np.exp(cross_entropy)

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not str